## todo

- [ ] meilleur filtre à l'entrée
    - les tags de suivi facebook divise les entrées d'une même page
    - exfiltrer les `login` et `next=`

In [ ]:
year = 2018

In [ ]:
import requests
import pandas as pd

In [ ]:
def get_datasets(site_id):

    endpoint = 'https://stats.data.gouv.fr/'
    
    params = {
        'idSite': site_id,
        'module': 'API',
        'method': 'Actions.getPageUrls',
        'format': 'json',
        'depth': 4,
        'expanded': 1,
        'flat': 1,
        'period': 'range',
        'date': '%s-01-01,%s-12-31' % (year,year),
        'filter_pattern': '^/fr/datasets/[a-zA-Z-]+/',
        'filter_limit': -1
#        'filter_truncate': False
    }
        
    r = requests.get(endpoint, params=params)
    
    print(r.url)
    
    df = pd.DataFrame(r.json())
    
    print(len(df))
    
    # <CLEAN> Remove total of `fr/datasets/`
    df.sort_values(by=['nb_hits'], inplace=True, ascending=False)
    df = df.iloc[1:]
    
    return df

df = get_datasets(1)

In [ ]:
def remove_trail(row):
    
    label = row["label"]
    clean = "/".join(label.split("/")[:4])+"/"
    
    return clean.lower()

df["label_clean"] = df.apply(remove_trail, axis=1)
                             
len(df[ df["label"] == df["label_clean"] ])

In [ ]:
df_clean = df[["nb_hits","label_clean"]].groupby("label_clean").sum().reset_index()

df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
df_clean.to_csv("out/datasets-hits-%s.csv" % year)